In [1]:
! pip install neo4j

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   -- ------------------------------------- 20.5/294.6 kB ? eta -:--:--
   ----- --------------------------------- 41.0/294.6 kB 495.5 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/294.6 kB 544.7 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/294.6 kB 544.7 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/294.6 kB 544.7 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/294.6 kB 544.7 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/294.6 kB 344.8 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/294.6 kB 344.8 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/294.6 kB 344.8 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/294.6 kB 344.8 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/294.6 kB 266.2 kB/s eta 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Configuration
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to create an employee node
def create_employee(employee_id, name, title):
    with driver.session() as session:
        session.run(
            "CREATE (e:Employee {employee_id: $employee_id, name: $name, title: $title})",
            employee_id=employee_id, name=name, title=title
        )
        print(f"Employee {name} created.")

# Function to create a department node
def create_department(department_id, name):
    with driver.session() as session:
        session.run(
            "CREATE (d:Department {department_id: $department_id, name: $name})",
            department_id=department_id, name=name
        )
        print(f"Department {name} created.")

# Function to assign an employee to a department
def assign_employee_to_department(employee_id, department_id):
    with driver.session() as session:
        session.run(
            """
            MATCH (e:Employee {employee_id: $employee_id})
            MATCH (d:Department {department_id: $department_id})
            CREATE (e)-[:WORKS_IN]->(d)
            """,
            employee_id=employee_id, department_id=department_id
        )
        print(f"Employee {employee_id} assigned to department {department_id}.")

# Example Usage:
create_employee("E001", "Alice", "Software Engineer")
create_department("D001", "Engineering")
assign_employee_to_department("E001", "D001")


Employee Alice created.
Department Engineering created.
Employee E001 assigned to department D001.


In [2]:
def get_employees_in_department(department_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {department_id: $department_id})
            RETURN e.employee_id AS employee_id, e.name AS name, e.title AS title
            """,
            department_id=department_id
        )
        employees = [{"employee_id": record["employee_id"], "name": record["name"], "title": record["title"]}
                    for record in result]
    return employees

# Example Usage:
employees = get_employees_in_department("D001")
print("Employees in Engineering Department:", employees)


Employees in Engineering Department: [{'employee_id': 'E001', 'name': 'Alice', 'title': 'Software Engineer'}]


In [3]:
def update_employee_department(employee_id, new_department_id):
    with driver.session() as session:
        session.run(
            """
            MATCH (e:Employee {employee_id: $employee_id})-[r:WORKS_IN]->(d:Department)
            DELETE r
            WITH e
            MATCH (new_d:Department {department_id: $new_department_id})
            CREATE (e)-[:WORKS_IN]->(new_d)
            """,
            employee_id=employee_id, new_department_id=new_department_id
        )
        print(f"Employee {employee_id} moved to department {new_department_id}.")

# Example Usage:
update_employee_department("E001", "D002")


Employee E001 moved to department D002.


In [4]:
def delete_employee(employee_id):
    with driver.session() as session:
        session.run(
            "MATCH (e:Employee {employee_id: $employee_id}) DETACH DELETE e",
            employee_id=employee_id
        )
        print(f"Employee {employee_id} deleted.")

# Example Usage:
delete_employee("E001")


Employee E001 deleted.
